# Import and pre-definitions

In [1]:
import os
MAIN_PATH = r'/home/luis-felipe'
DATA_PATH = r'/home/luis-felipe/data'
PATH_MODELS = os.path.join(MAIN_PATH,'torch_models')
RESULTS_PATH = os.path.join(MAIN_PATH,'results')
FIGS_PATH = os.path.join(MAIN_PATH,'results','figs')

In [2]:
import torch
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [3]:
# Define o computador utilizado como cuda (gpu) se existir ou cpu caso contrário
print(torch.cuda.is_available())
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
SEED = 42
#torch.set_default_dtype(torch.float64)
torch.manual_seed(SEED)
np.random.seed(SEED)

True


In [4]:
import sys
sys.path.insert(1, '..')
sys.path.insert(1, '../..')

import models
from utils import measures,metrics
from data_utils import upload_logits,split_data
import post_hoc

# Evaluate logits

In [5]:
METRIC = metrics.AURC#lambda x,y: 1-metrics.AUROC(x,y)#metrics.AURC
DATASET = 'ImageNet'

In [6]:
corruptions = os.listdir(os.path.join(DATA_PATH,DATASET,'corrupted'))
lvls = range(1,6)

In [6]:
extras = ['speckle_noise', 'gaussian_blur','spatter','saturate']
for c in extras:
    corruptions.remove(c)
print(len(corruptions))


KeyboardInterrupt



In [ ]:
d_sac = {c:[] for c in corruptions}
d_sac_T = {c:[] for c in corruptions}
d_sac_baseline = {c:[] for c in corruptions}

acc = {c:[] for c in corruptions}

In [ ]:
VAL_SIZE = 0.1
for model_arc in ['resnet50']:#models.list_models():#models_list:
    print(model_arc)
    logits,labels = split_data.split_logits(*upload_logits(model_arc,DATASET,PATH_MODELS, 
                                    split = 'test', device = dev),VAL_SIZE,SEED)[:2]
    risk = measures.wrong_class(logits,labels).float()
    acc_baseline = 0.8086#1-risk.mean().item()
    for corruption in corruptions:
        for lvl in lvls:
            with torch.no_grad():
                
                logits_c,labels_c = split_data.split_logits(*upload_logits(model_arc,DATASET,PATH_MODELS, 
                                        split = ('corrupted',corruption,str(lvl)), device = dev, data_dir = DATA_PATH),VAL_SIZE,SEED)[2:]
                
                risk_c = measures.wrong_class(logits_c,labels_c).float()
            
                logits_c = post_hoc.centralize(logits_c)
                logits = post_hoc.centralize(logits)
                #except: 
                #    print('não deu')
                #    continue
                acc[corruption].append(1-risk_c.mean().item())
                p = post_hoc.optimize.p(logits,risk,measures.max_logit,METRIC)
                T = post_hoc.optimize.T(logits,risk,measures.MSP,METRIC)
                d_sac[corruption].append(metrics.SAC(risk_c,post_hoc.MaxLogit_p(logits_c,p=p),acc_baseline))
                d_sac_T[corruption].append(metrics.SAC(risk_c,measures.MSP(logits_c.div(T)),acc_baseline))
                d_sac_baseline[corruption].append(metrics.SAC(risk_c,measures.MSP(logits_c),acc_baseline))

In [ ]:
sac = []
sac_T = []
sac_baseline = []
for c in corruptions:
    sac.append(d_sac[c])
    sac_T.append(d_sac_T[c])
    sac_baseline.append(d_sac_baseline[c])
sac = np.array(sac)
sac_T = np.array(sac_T)
sac_baseline = np.array(sac_baseline)

In [ ]:
sac.mean(0)

In [ ]:
corruptions